<a href="https://colab.research.google.com/github/ciepielajan/SDA_SMOG_Wawelski/blob/main/Jan/SDA_2021_04_17_Projekt_koncowy_pipeline_column_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 Załadowanie plików







In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
!gdown --id "186S03hFJ67K4eV1BWb2_6lZxAYG7dBM4"

Downloading...
From: https://drive.google.com/uc?id=186S03hFJ67K4eV1BWb2_6lZxAYG7dBM4
To: /content/archive.zip
2.39MB [00:00, 75.7MB/s]


In [ ]:
!unzip -q "archive.zip" -d data/

In [ ]:
all_file_names = glob.glob("data/*2017.csv")
all_file_names = sorted(all_file_names)
for counter, file in enumerate(all_file_names):
    print(counter, file)

0 data/april-2017.csv
1 data/august-2017.csv
2 data/december-2017.csv
3 data/february-2017.csv
4 data/january-2017.csv
5 data/july-2017.csv
6 data/june-2017.csv
7 data/march-2017.csv
8 data/may-2017.csv
9 data/november-2017.csv
10 data/october-2017.csv
11 data/september-2017.csv


In [ ]:
jan = pd.read_csv(all_file_names[4])
feb = pd.read_csv(all_file_names[3])
mar = pd.read_csv(all_file_names[7])
apr = pd.read_csv(all_file_names[0])
may = pd.read_csv(all_file_names[8])
jun = pd.read_csv(all_file_names[6])
jul = pd.read_csv(all_file_names[5])
aug = pd.read_csv(all_file_names[1])
sep = pd.read_csv(all_file_names[11])
octo = pd.read_csv(all_file_names[10])
nov = pd.read_csv(all_file_names[9])
dec = pd.read_csv(all_file_names[2])

In [ ]:
dec.head(1)

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-12-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,96,100542,39,38,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.0,70.0,100479.0,25.0,23.0,40.0,0.0,72.0,100527.0,29.0,28.0,48.0,0,96,100531,...,100616.0,27.0,25.0,41.0,0.0,97.0,100602.0,25.0,21.0,35.0,0,97,100810,34,33,62,5.0,97.0,100036.0,12.0,10.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,0,88,100554,32,33,74,0,94,100607,34,36,66


In [ ]:
sensors = pd.read_csv("data/sensor_locations.csv")

In [ ]:
sensors[:3]

,id,latitude,longitude
0,3,50.057748,19.961375
1,140,50.057747,19.961374
2,142,50.084816,19.878066


2 Połączenie wszystkich df


In [ ]:
year = pd.concat([jan, feb, mar, apr, may, jun, jul, aug, sep, octo, nov, dec])

In [ ]:
year.shape

(8593, 337)

In [ ]:
year.head(2)

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-01-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,68.0,102279.0,115.0,127.0,196.0,-3.0,57.0,102505.0,196.0,222.0,346.0,0.0,49.0,102689.0,124.0,145.0,229.0,2.0,25.0,102379.0,148.0,176.0,277.0,1.0,27.0,102437.0,...,102253.0,184.0,218.0,322.0,0.0,54.0,101672.0,137.0,153.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01T01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,68.0,102228.0,105.0,116.0,180.0,-3.0,56.0,102454.0,187.0,211.0,330.0,0.0,49.0,102635.0,124.0,145.0,230.0,1.0,25.0,102325.0,131.0,156.0,247.0,0.0,27.0,102387.0,...,102201.0,146.0,169.0,250.0,0.0,52.0,101623.0,119.0,132.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3 Transformacja df







In [ ]:
year["UTC time"] = pd.to_datetime(year["UTC time"], format="%Y-%m-%dT%H:%M:%S")
year = year.set_index("UTC time")
year[:3]

,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,171_pm1,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
UTC time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,68.0,102279.0,115.0,127.0,196.0,-3.0,57.0,102505.0,196.0,222.0,346.0,0.0,49.0,102689.0,124.0,145.0,229.0,2.0,25.0,102379.0,148.0,176.0,277.0,1.0,27.0,102437.0,147.0,...,102253.0,184.0,218.0,322.0,0.0,54.0,101672.0,137.0,153.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,68.0,102228.0,105.0,116.0,180.0,-3.0,56.0,102454.0,187.0,211.0,330.0,0.0,49.0,102635.0,124.0,145.0,230.0,1.0,25.0,102325.0,131.0,156.0,247.0,0.0,27.0,102387.0,129.0,...,102201.0,146.0,169.0,250.0,0.0,52.0,101623.0,119.0,132.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,66.0,102149.0,110.0,121.0,189.0,-4.0,55.0,102375.0,187.0,212.0,333.0,0.0,49.0,102558.0,108.0,125.0,201.0,1.0,24.0,102244.0,108.0,127.0,203.0,1.0,27.0,102310.0,134.0,...,102119.0,138.0,160.0,237.0,0.0,49.0,101536.0,97.0,105.0,162.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
new_df = pd.DataFrame()

for i in year.groupby(year.columns.str.split('_').str[0], axis=1):
  old = i[1].columns
  new = i[1].columns.str.split("_").str[1]
  zmiana = { k:v  for k,v in zip(old,new)}
  i[1].rename(columns=zmiana,inplace=True)
  i[1]["id"] = i[0]
  new_df = new_df.append(i[1])
  # print(i[1])

In [ ]:
new_df[:3]

,temperature,humidity,pressure,pm1,pm25,pm10,id
UTC time,,,,,,,
2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140


3.2 Połaczenie wartości z konkretnymi lokalizacjami wg ID sensoru

In [ ]:
new_df["id"] = new_df["id"].astype(int)
sensors["id"] = sensors["id"].astype(int)

In [ ]:
new_df = new_df.reset_index()

In [ ]:
geo_df = new_df.merge(sensors, how='left', on='id')
geo_df[:3]

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id,latitude,longitude
0,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374


4 EDA

In [ ]:
geo_df.shape

(481208, 10)

4.1. Ile jest Nan w danych kolumnach

In [ ]:
geo_df.isnull().sum()

UTC time            0
temperature    109558
humidity       109638
pressure       172529
pm1             92854
pm25            92835
pm10            92832
id                  0
latitude            0
longitude           0
dtype: int64

4.2. Czy można znaleźć brakujące dane lub czymś je zastąpić

In [ ]:
geo_df = geo_df.dropna()
geo_df.shape

(291754, 10)

`po usunięciu indexów resetuje je aby ponownie miały ciągłość`

In [ ]:
geo_df.reset_index(drop=True, inplace=True)
geo_df.index

RangeIndex(start=0, stop=291754, step=1)

4.3. Jakie zastosować normy i ich zakresy dla stężenia pm10 w celu określenia jakości powietrza

In [ ]:
geo_df.pm10.describe([0.4,0.75, 0.9, 0.95,0.99])

count    291754.000000
mean         49.589233
std          53.379528
min          -1.000000
40%          26.000000
50%          34.000000
75%          61.000000
90%         102.000000
95%         154.000000
99%         282.000000
max         664.000000
Name: pm10, dtype: float64

Wnioski
> podejrzana wartość to `min = -1`. Poziom stężenia nie może być ujemny. Dane prawdopodobnie wymagają dalszego oczyszczenia. 

> niepokojąca jest też wartość `max=664` . Normy zaczynają sie od 25 a kończna na 150 .  Czy dla wartości powyżej 200 powinniśmy przyjąć własne normy ?

> Istotny jest fakt że 99% danych mieści się w zakresie od 0 do 282.  

Wnioski:
> Źródła zgodnie podają że stan powyżej 100 dla pm10 jest "alarmujący". Z uwagi że podaczas wysiku fizycznego na świerzym powietrzu pobiera się wiecej powietrza niż przy zwykłej aktywnośći uznaję że wszytkie pomiary powyżej 100 będą uznawane za szkodliwe dla zdrowia. 

> Zakresy dla PM10 które będę w dalszych krokach prowadziło to 0,25,50,100,150

5 Zmiana danych na kategoryczne - Rozbicie TimeSeries na dzień, miesiąc etc


In [ ]:
import warnings
warnings.filterwarnings("ignore")

Zamieniamy UTC Time na dane kategoryczne \
`lato = okres nie grzewczy` \
`zima = okres grzewczy` \
`dzien = pora dnia nie grzewcza ( ludzi sa w pracy) od 10-16` \
`noc = pora dnia grzewcza ( ludzi wracają do domow (korki) i grzanie przed nocą ) od 17-9`

In [ ]:
def parse_season(x):
    if x in [4,5,6,7,8,9]:
       return "lato"
    elif x in [1,2,3,10,11,12]:
       return "zima"
    else:
       return "bład"

In [ ]:
def parse_hours(x):
    if x in [10,11,12,13,14,15,16]:
       return "dzien"
    elif x in [17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,]:
       return "noc"
    else:
       return "bład"

In [ ]:
geo_df["season"] = geo_df["UTC time"].dt.month.apply(parse_season)
geo_df["part_of_the_day"] = geo_df["UTC time"].dt.hour.apply(parse_hours)
geo_df[:3]

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id,latitude,longitude,season,part_of_the_day
0,2017-03-17 11:00:00,12.0,44.0,101244.0,16.0,10.0,22.0,140,50.057747,19.961374,zima,dzien
1,2017-03-17 12:00:00,12.0,45.0,101147.0,18.0,12.0,26.0,140,50.057747,19.961374,zima,dzien
2,2017-03-17 13:00:00,12.0,43.0,101049.0,18.0,12.0,26.0,140,50.057747,19.961374,zima,dzien


In [ ]:
geo_df.shape

(291754, 12)

`ustalenie X i y`

In [ ]:
X = geo_df[["latitude","longitude","season","part_of_the_day"]]
X[:3]

,latitude,longitude,season,part_of_the_day
0,50.057747,19.961374,zima,dzien
1,50.057747,19.961374,zima,dzien
2,50.057747,19.961374,zima,dzien


In [ ]:
y = geo_df["pm10"]
y[:3]

0    22.0
1    26.0
2    26.0
Name: pm10, dtype: float64

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# define the data preparation for the columns
t = [
     ('ohe', OneHotEncoder(drop="first"), ["season","part_of_the_day"]),  #na kategorycznych zastosuj OneHotEncoder
     ]
col_transform = ColumnTransformer(transformers=t, remainder="passthrough")   #wykonaj transformacje . parametr remainder definiuje co zrobić z pozostałmi kolumnami
col_transform

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('ohe',
                                 OneHotEncoder(categories='auto', drop='first',
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=True),
                                 ['season', 'part_of_the_day'])],
                  verbose=False)

In [ ]:
# ohe = OneHotEncoder(drop="first") 
# geo_df_trans = ohe.fit_transform(geo_df[["season","part_of_the_day"]])

In [ ]:
# geo_df_trans = pd.DataFrame(geo_df_trans.toarray(),columns=ohe.get_feature_names())
# geo_df_trans[:3]

,x0_zima,x1_noc
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [ ]:
# #podgląd czy po transformacji wszystko sie zgadza
# geo_df = pd.concat([geo_df,geo_df_trans],axis=1)
# geo_df[:3]

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id,latitude,longitude,season,part_of_the_day,x0_zima,x1_noc
0,2017-03-17 11:00:00,12.0,44.0,101244.0,16.0,10.0,22.0,140,50.057747,19.961374,zima,dzien,1.0,0.0
1,2017-03-17 12:00:00,12.0,45.0,101147.0,18.0,12.0,26.0,140,50.057747,19.961374,zima,dzien,1.0,0.0
2,2017-03-17 13:00:00,12.0,43.0,101049.0,18.0,12.0,26.0,140,50.057747,19.961374,zima,dzien,1.0,0.0


In [ ]:
# geo_df.shape

(291754, 14)

6 Normalizacja współrzędnych - Standard scaller lub wzór (-50x1000)

7 Predykcja - szukanie najlepszego modelu

`ustawienie X i y`

In [ ]:
# X = geo_df[["latitude","longitude","x0_zima","x1_noc"]]
# X[:3]

,latitude,longitude,x0_zima,x1_noc
0,50.057747,19.961374,1.0,0.0
1,50.057747,19.961374,1.0,0.0
2,50.057747,19.961374,1.0,0.0


In [ ]:
# y = geo_df["pm10"]
# y[:3]

0    22.0
1    26.0
2    26.0
Name: pm10, dtype: float64

7.1 Regression

7.1.1 Regresja liniowa

In [ ]:
from sklearn.linear_model import LinearRegression

7.1.1.2 Regresja liniowa Podstawowy model

In [ ]:
model_LinearRegression = LinearRegression()
# model.fit(X, y)

In [ ]:
pipeline_LinearRegression = Pipeline(steps=[('prep',col_transform), ('model', model_LinearRegression)])   # połącz  transformację kolumn z trenowaniem modelu 
pipeline_LinearRegression.fit(X,y)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['season',
                                                   'part_of_the_day'])],
                                   verbose=False)),
                ('model',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jo

In [ ]:
pipeline_LinearRegression.score(X, y)

0.19745994534646116

In [ ]:
#podgląd wszysktich danych testowych danych
podglad = geo_df.iloc[[3,1812,4000,5994]]
#przygotowanie dandych do modelu
df_predict = podglad[X.columns]
df_predict

,latitude,longitude,season,part_of_the_day
3,50.057747,19.961374,zima,dzien
1812,50.057747,19.961374,lato,dzien
4000,50.057747,19.961374,lato,noc
5994,50.057747,19.961374,zima,noc


In [ ]:
#predykcja
# predict = model.predict(df_predict)
# predict

In [ ]:
# pipeline  zrobi transformacje odpowiednich kolumn i zastuje odpowiedni model
predict = pipeline_LinearRegression.predict(df_predict)
predict

array([58.08787322, 13.31758695, 30.41229151, 75.18257777])

In [ ]:
#porównanie 
podglad["pm10"].values[0]
podglad["predict_pm10"] = predict
podglad[X.columns.to_list() + ["pm10","predict_pm10"] ]

,latitude,longitude,season,part_of_the_day,pm10,predict_pm10
3,50.057747,19.961374,zima,dzien,32.0,58.087873
1812,50.057747,19.961374,lato,dzien,6.0,13.317587
4000,50.057747,19.961374,lato,noc,11.0,30.412292
5994,50.057747,19.961374,zima,noc,60.0,75.182578


7.1.1.3 Regresja liniowa - Dostrajanie modelu np grid search

7.1.2.1 RandomForestRegressor()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

7.1.2.2 RandomForestRegressor() - Podstawowy model

In [ ]:
model_RandomForestRegressor = RandomForestRegressor()
# model.fit(X, y)

In [ ]:
pipeline_RandomForestRegressor = Pipeline(steps=[('prep',col_transform), ('model', model_RandomForestRegressor)])   # połącz  transformację kolumn z trenowaniem modelu 
pipeline_RandomForestRegressor.fit(X,y)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['season',
                                                   'part_of_the_day'])],
                                   verbose=False)),
                ('model',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
 

In [ ]:
pipeline_RandomForestRegressor.score(X, y)

0.23145944244328365

In [ ]:
#podgląd wszysktich danych testowych danych
podglad = geo_df.iloc[[3,1812,4000,5994]]
#przygotowanie dandych do modelu
df_predict = podglad[X.columns]
df_predict

,latitude,longitude,season,part_of_the_day
3,50.057747,19.961374,zima,dzien
1812,50.057747,19.961374,lato,dzien
4000,50.057747,19.961374,lato,noc
5994,50.057747,19.961374,zima,noc


In [ ]:
#predykcja
predict = pipeline_RandomForestRegressor.predict(df_predict)
predict

array([49.79283171, 21.09140096, 35.80386666, 71.96246107])

In [ ]:
#porównanie 
podglad["pm10"].values[0]
podglad["predict_pm10"] = predict
podglad[X.columns.to_list() + ["pm10","predict_pm10"] ]

,latitude,longitude,season,part_of_the_day,pm10,predict_pm10
3,50.057747,19.961374,zima,dzien,32.0,49.792832
1812,50.057747,19.961374,lato,dzien,6.0,21.091401
4000,50.057747,19.961374,lato,noc,11.0,35.803867
5994,50.057747,19.961374,zima,noc,60.0,71.962461


7.1.2.3 RandomForestRegressor() - Dostrajanie modelu np grid search

In [ ]:
# {'bootstrap': [True, False],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [ ]:
param_grid = {
    'bootstrap': [True, False],
    # 'max_depth': [2,5,10,50,100, None],
    # 'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5, 10],
    }

In [ ]:
from sklearn.model_selection import GridSearchCV

model_RandomForestRegressor_GSCV = RandomForestRegressor()

pipeline_RandomForestRegressor_GSCV = Pipeline(steps=[('prep',col_transform), ('model', model_RandomForestRegressor_GSCV)])   # połącz  transformację kolumn z trenowaniem modelu 
grid_search = GridSearchCV(pipeline_RandomForestRegressor_GSCV, param_grid, cv=3, n_jobs = -1, verbose=10)
# grid_search.fit(X,y)

In [ ]:
# grid_search.best_params_

In [ ]:
# model = RandomForestRegressor(**grid_search.best_params_)
# model.fit(X, y)

7.1.3.1 DecisionTreeRegressor  - Podstawowy model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model_DecisionTreeRegressor = DecisionTreeRegressor()

In [ ]:
pipeline_DecisionTreeRegressor = Pipeline(steps=[('prep',col_transform), ('model', model_DecisionTreeRegressor)])   # połącz  transformację kolumn z trenowaniem modelu 
pipeline_DecisionTreeRegressor.fit(X,y)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['season',
                                                   'part_of_the_day'])],
                                   verbose=False)),
                ('model',
                 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',


In [ ]:
pipeline_DecisionTreeRegressor.score(X, y)

0.2314634895937573

In [ ]:
#podgląd wszysktich danych testowych danych
podglad = geo_df.iloc[[3,1812,4000,5994]]
#przygotowanie dandych do modelu
df_predict = podglad[X.columns]
df_predict

,latitude,longitude,season,part_of_the_day
3,50.057747,19.961374,zima,dzien
1812,50.057747,19.961374,lato,dzien
4000,50.057747,19.961374,lato,noc
5994,50.057747,19.961374,zima,noc


In [ ]:
#predykcja
predict = pipeline_DecisionTreeRegressor.predict(df_predict)
predict

array([49.72332016, 21.08879668, 35.81060342, 71.93603062])

In [ ]:
#porównanie 
podglad["pm10"].values[0]
podglad["predict_pm10"] = predict
podglad[X.columns.to_list() + ["pm10","predict_pm10"] ]

,latitude,longitude,season,part_of_the_day,pm10,predict_pm10
3,50.057747,19.961374,zima,dzien,32.0,49.723320
1812,50.057747,19.961374,lato,dzien,6.0,21.088797
4000,50.057747,19.961374,lato,noc,11.0,35.810603
5994,50.057747,19.961374,zima,noc,60.0,71.936031


8 Zestawienie wyników predykcji - wybranie najlepszego modelu do predykcji 

9 Zapisanie plików niezbędnych do zastosowania modelu na produkcji



In [ ]:
import pickle
with open('pipeline.pickle', 'wb') as handle:
    pickle.dump(pipeline_RandomForestRegressor, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del pipeline

10 Prototyp

10.1 Utworzenie siatki punktów w na terenie całego krakowa zlokalizowanych co 500m

In [ ]:
left_top_corner =  [50.107 ,19.82]

points = pd.DataFrame()

for i in range(45):
  id_i = str(i+1) 
  i = i * 0.00463  # im mniejsza wartość tym bliższe odległości od kolejnego punktu
  for x in range(35):
    id = id_i + "-" + str(x+1)
    x = x * 0.00297      # im mniejsza wartość tym bliższe odległości od kolejnego punktu
    lat = left_top_corner[0] - x
    lon = left_top_corner[1] + i
    points = points.append({"id":id,"latitude":lat,"longitude":lon},ignore_index=True)

points

,id,latitude,longitude
0,1-1,50.10700,19.82000
1,1-2,50.10403,19.82000
2,1-3,50.10106,19.82000
3,1-4,50.09809,19.82000
4,1-5,50.09512,19.82000
...,...,...,...
1570,45-31,50.01790,20.02372
1571,45-32,50.01493,20.02372
1572,45-33,50.01196,20.02372
1573,45-34,50.00899,20.02372


10.2 Obliczenie predykcji dla wszystkich stacji pomiarowych dla konkretnej daty

In [ ]:
#podgląd danych do testowej predykcji
geo_df.iloc[1812].to_frame().T

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id,latitude,longitude,season,part_of_the_day
1812,2017-06-03 11:00:00,24,37,101792,4,3,6,140,50.0577,19.9614,lato,dzien


`wprowadzenie danych przez użytkownika`

In [ ]:
data = "2017-12-03"
time = "11"
user_time = pd.Timestamp(data + " " + time + ":00:00")
user_latitude = "50.057747"
user_longitude = "19.961374"

In [ ]:
user_date = pd.DataFrame([user_latitude, user_longitude]).T
user_date.columns = ["latitude","longitude"]
user_date["season"] = parse_season(user_time.month)
user_date["part_of_the_day"] = parse_hours(user_time.hour)
user_date

,latitude,longitude,season,part_of_the_day
0,50.057747,19.961374,zima,dzien


`do dodatkowych punktów dodaje dane po transformacji`

In [ ]:
points_prediction = points[["latitude","longitude"]]
points_prediction["season"] = user_date["season"].values[0]
points_prediction["part_of_the_day"] = user_date["part_of_the_day"].values[0]
points_prediction[:3]

,latitude,longitude,season,part_of_the_day
0,50.10700,19.82,zima,dzien
1,50.10403,19.82,zima,dzien
2,50.10106,19.82,zima,dzien


In [ ]:
#predykcja
predict = pipeline.predict(points_prediction)
predict

array([52.19840258, 52.19840258, 51.17661215, ..., 45.9599571 ,
       45.9599571 , 45.9599571 ])

In [ ]:
points_prediction["prediction_pm10"] = predict
points_prediction[:3]

,latitude,longitude,season,part_of_the_day,prediction_pm10
0,50.10700,19.82,zima,dzien,52.198403
1,50.10403,19.82,zima,dzien,52.198403
2,50.10106,19.82,zima,dzien,51.176612


In [ ]:
points_prediction["prediction_pm10"].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
prediction_pm10,1575.0,53.641153,8.753608,37.92413,47.375972,52.271497,57.722245,91.716763


In [ ]:
!pip -q install folium
import folium

In [ ]:
location = [50.0614 ,19.9366]

m = folium.Map(location=location, zoom_start=17)


for ind in points_prediction.index: 
    tooltip_txt = 'PM10:' + str(points_prediction['prediction_pm10'][ind])
    
    if points_prediction['prediction_pm10'][ind] <= 25:
      color="green",
    elif (points_prediction['prediction_pm10'][ind] > 25) & (points_prediction['prediction_pm10'][ind] <= 50):
      color="orange",
    elif (points_prediction['prediction_pm10'][ind] > 50) & (points_prediction['prediction_pm10'][ind] <= 100):
      color="red",
    elif (points_prediction['prediction_pm10'][ind] > 100):
      color="black",
    else:
      color="white"

    folium.Circle(
          location=[float(points_prediction['latitude'][ind]), float(points_prediction['longitude'][ind])], 
          radius=165,
          # popup="Laurelhurst Park",
          # fill=True,
          # fill_color="#3186cc",
          fill_color=color,
          color=color,
          tooltip= tooltip_txt
      ).add_to(m)
m

In [ ]:
name_file = "index.html"

m.save(name_file)
from google.colab import files
files.download(name_file) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

10.3 Utworzenie modelu K-najbliższych sąsiadów -regression (odległość, haversine distance- odległość km )